**Importacion de librerias**

In [1]:
import pandas as pd
import numpy as np

**Importacion y union de datos**

In [2]:
#Leemos los csv en forma de pandas DataFrame 
amazon = pd.read_csv("MLOpsReviews/amazon_prime_titles.csv")
disney = pd.read_csv("MLOpsReviews/disney_plus_titles.csv")
hulu = pd.read_csv("MLOpsReviews/hulu_titles.csv")
netflix = pd.read_csv("MLOpsReviews/netflix_titles.csv")

### **ID***

Segun la consigna estandarizamos los ID de rating con los de las plataformas

In [3]:
#estandarizamos las ID con las tablas de valoraciones para poder hacer filtraciones y crear una tabla con todos los datos unidos
amazon.show_id = amazon.show_id.replace({'s':'as'}, regex=True)
disney.show_id = disney.show_id.replace({'s':'ds'}, regex=True)
hulu.show_id = hulu.show_id.replace({'s':'hs'}, regex=True)
netflix.show_id = netflix.show_id.replace({'s':'ns'}, regex=True)

Unimos todos los datos para tener un dataframe global

In [4]:
#unimos todos los datos en una tabla data
data = pd.concat([amazon, disney, hulu, netflix])

In [19]:
data.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Valoraciones_x,Promedio_Valoraciones_x,Valoraciones_y,Promedio_Valoraciones_y,Valoraciones,Promedio_Valoraciones
0,as7362,Movie,"Drunk, Stoned, Brilliant, Dead: The Story of t...",Douglas Tirola,"Chevy Chase, Judd Apatow, Henry Beard, PJ O’Ro...",NaN,NaN,2015,R,95 min,Documentary,"From 1970s-1990s, the hippest and most outrage...",469,3.7,469,3.7,469,3.7
1,ns3234,Movie,Pranaam,Sanjiv Jaiswal,"Rajeev Khandelwal, Samiksha Singh, S.M. Zaheer...",India,"November 25, 2019",2019,TV-14,117 min,"Action & Adventure, Dramas, International Movies",Aspiring to fulfill his father’s dream and bec...,490,3.7,490,3.7,490,3.7


### **Spark**

Creamos una app spark para poder manipular grandes volumenes de datos

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('app').getOrCreate()

Leemos todos los CSV con spark

In [6]:
#Leemos estas tablas con spark por su tamaño
d1 = spark.read.csv("MLOpsReviews/ratings/1.csv", header=True, inferSchema=True)
d2 = spark.read.csv("MLOpsReviews/ratings/2.csv", header=True, inferSchema=True)
d3 = spark.read.csv("MLOpsReviews/ratings/3.csv", header=True, inferSchema=True)
d4 = spark.read.csv("MLOpsReviews/ratings/4.csv", header=True, inferSchema=True)
d5 = spark.read.csv("MLOpsReviews/ratings/5.csv", header=True, inferSchema=True)
d6 = spark.read.csv("MLOpsReviews/ratings/6.csv", header=True, inferSchema=True)
d7 = spark.read.csv("MLOpsReviews/ratings/7.csv", header=True, inferSchema=True)
d8 = spark.read.csv("MLOpsReviews/ratings/8.csv", header=True, inferSchema=True)


Unimos todos los CSV en un solo dataframe de ratings

In [7]:
#Unimos los datos rating
df_list = [d1, d2, d3, d4, d5, d6, d7, d8]

rating = df_list[0]
for i in range(1, len(df_list)):
    rating = rating.union(df_list[i])
    
rating.limit(3).show()

+------+------+----------+-------+
|userId|rating| timestamp|movieId|
+------+------+----------+-------+
|     1|   1.0|1425941529|  as680|
|     1|   4.5|1425942435| ns2186|
|     1|   5.0|1425941523| hs2381|
+------+------+----------+-------+



Creacion de datos sobre el promedio y la cantidad de valoraciones segun cada ID

In [9]:
#Contamos cada valoracion por ID para sacar el numero de valoraciones por ID
d1c = d1.groupBy("movieId").agg(count("movieId").alias("count"))
d2c = d2.groupBy("movieId").agg(count("movieId").alias("count2"))
d3c = d3.groupBy("movieId").agg(count("movieId").alias("count3"))
d4c = d4.groupBy("movieId").agg(count("movieId").alias("count4"))
d5c = d5.groupBy("movieId").agg(count("movieId").alias("count5"))
d6c = d6.groupBy("movieId").agg(count("movieId").alias("count6"))
d7c = d7.groupBy("movieId").agg(count("movieId").alias("count7"))
d8c = d8.groupBy("movieId").agg(count("movieId").alias("count8"))

#sacamos el promedio de rating por cada ID
d1p = d1.groupBy("movieId").agg(round(avg("rating"), 2).alias("prom"))
d2p = d2.groupBy("movieId").agg(round(avg("rating"), 2).alias("prom2"))
d3p = d3.groupBy("movieId").agg(round(avg("rating"), 2).alias("prom3"))
d4p = d4.groupBy("movieId").agg(round(avg("rating"), 2).alias("prom4"))
d5p = d5.groupBy("movieId").agg(round(avg("rating"), 2).alias("prom5"))
d6p = d6.groupBy("movieId").agg(round(avg("rating"), 2).alias("prom6"))
d7p = d7.groupBy("movieId").agg(round(avg("rating"), 2).alias("prom7"))
d8p = d8.groupBy("movieId").agg(round(avg("rating"), 2).alias("prom8"))

d1c.limit(3).show(), d1p.limit(3).show()

+-------+-----+
|movieId|count|
+-------+-----+
| ns8641|   65|
| as1586|   73|
| hs1299|   69|
+-------+-----+

+-------+----+
|movieId|prom|
+-------+----+
| ns8641|3.49|
| as1586|3.34|
| hs1299|3.61|
+-------+----+



(None, None)

In [10]:
#Unimos las cuentas para sumarlas
dfs = [d2c, d3c, d4c, d5c, d6c, d7c, d8c]
for df in dfs:
    d1c = d1c.join(df, on="movieId", how="outer").fillna(0)  
    
dfs = [d2p, d3p, d4p, d5p, d6p, d7p, d8p]
for df in dfs:
    d1p = d1p.join(df, on="movieId", how="outer").fillna(0)   

d1c.limit(3).show(), d1p.limit(3).show()

+-------+-----+------+------+------+------+------+------+------+
|movieId|count|count2|count3|count4|count5|count6|count7|count8|
+-------+-----+------+------+------+------+------+------+------+
|    as1|   59|    53|    58|    75|    79|    83|    26|    69|
|   as10|   75|    70|    65|    64|    71|    75|    29|    64|
|  as100|   52|    52|    66|    79|    57|    46|    17|    61|
+-------+-----+------+------+------+------+------+------+------+

+-------+----+-----+-----+-----+-----+-----+-----+-----+
|movieId|prom|prom2|prom3|prom4|prom5|prom6|prom7|prom8|
+-------+----+-----+-----+-----+-----+-----+-----+-----+
|    as1| 3.5| 3.55| 3.44| 3.46| 3.46| 3.43| 3.38| 3.49|
|  as100|3.38| 3.52| 3.42| 3.55| 3.76| 3.79| 4.09| 3.75|
| as1002|3.91| 3.49| 3.45| 3.63| 3.59| 3.64| 3.77| 3.42|
+-------+----+-----+-----+-----+-----+-----+-----+-----+



(None, None)

### **Promedios y Numero de Valoraciones**

Spark funciona muy bien para manipular los millones de datos que teniamos antes, pero luego de las funciones de agregación, la cantidad de datos disminuyeron, por lo que conviene usar pandas

In [11]:
#como son menos las convertimos a formato pandas para agillizar el proceso de suma
ranking = d1c.toPandas()
#sumamos todas las columnas para quedarnos con una sola
ranking["Valoraciones"] = ranking.loc[:, ranking.columns != 'movieId'].sum(axis=1)
ranking = ranking[["movieId","Valoraciones"]]
ranking.columns = ['show_id', 'Valoraciones']

#como son menos las convertimos a formato pandas para agillizar el proceso de suma
ranking_promedio = d1p.toPandas()
#sumamos todas las columnas para quedarnos con una sola
ranking_promedio["Promedio_Valoraciones"] = ranking_promedio.loc[:, ranking_promedio.columns != 'movieId'].mean(axis=1).round(1)
ranking_promedio = ranking_promedio[["movieId","Promedio_Valoraciones"]]
ranking_promedio.columns = ['show_id', 'Promedio_Valoraciones']

ranking.head()

,show_id,Valoraciones
0,as1,502
1,as10,513
2,as100,430
3,as1000,485
4,as1001,469


In [12]:
ranking_promedio.head()

,show_id,Promedio_Valoraciones
0,as1,3.5
1,as100,3.7
2,as1002,3.6
3,as1006,3.5
4,as1008,3.4


Unimos estos nuevos datos que extraimos de rating y los metemos a nuestro dataframe global

In [18]:
#añadimos el numero de valoraciones para cada pelicula
data = pd.merge(data, ranking, on='show_id')
#añadimos el numero de valoraciones para cada pelicula
data = pd.merge(data, ranking_promedio, on='show_id')

data.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Valoraciones_x,Promedio_Valoraciones_x,Valoraciones_y,Promedio_Valoraciones_y,Valoraciones,Promedio_Valoraciones
0,as7362,Movie,"Drunk, Stoned, Brilliant, Dead: The Story of t...",Douglas Tirola,"Chevy Chase, Judd Apatow, Henry Beard, PJ O’Ro...",NaN,NaN,2015,R,95 min,Documentary,"From 1970s-1990s, the hippest and most outrage...",469,3.7,469,3.7,469,3.7


In [14]:
#Ordenamos descendientemente por el numero promedio de rating
data = data.sort_values(by = "Promedio_Valoraciones", ascending = False)
data.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Valoraciones,Promedio_Valoraciones
7361,as7362,Movie,"Drunk, Stoned, Brilliant, Dead: The Story of t...",Douglas Tirola,"Chevy Chase, Judd Apatow, Henry Beard, PJ O’Ro...",NaN,NaN,2015,R,95 min,Documentary,"From 1970s-1990s, the hippest and most outrage...",469,3.7
17424,ns3234,Movie,Pranaam,Sanjiv Jaiswal,"Rajeev Khandelwal, Samiksha Singh, S.M. Zaheer...",India,"November 25, 2019",2019,TV-14,117 min,"Action & Adventure, Dramas, International Movies",Aspiring to fulfill his father’s dream and bec...,490,3.7


### **Exportacion de Datos**

In [15]:
data.to_csv('Raw/data_raw.csv', index=False)